In [1]:
from dotenv import load_dotenv
from pathlib import Path
import sys
import os
load_dotenv()

BASE_DIR = Path.cwd().resolve().parent
ROOT_DIR = Path.cwd().parent

SQL_DB_PATH = BASE_DIR/'databases/BaseHop.db'
print(BASE_DIR)
sys.path.append(str(ROOT_DIR))

C:\Users\soulemane\Documents\techSante_v2


## Tools 

In [2]:
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_groq import ChatGroq
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage
from sources.prompts import SYSTEME_PROMPTE_TEST
import sqlite3

c:\Users\soulemane\Documents\techSante_v2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
@tool
def check_beds_availability():
    """Vérifie les lits disponibles par hôpital et par service."""
    # print("[DEBUG] L'agent consulte la base de données...")
    try:
        if not os.path.exists(SQL_DB_PATH):
            return 'Erreur : Base de données introuvable.'
        
        # Connexion 
        conn = sqlite3.connect(f"file:{SQL_DB_PATH}?mode=ro", uri=True)
        c = conn.cursor()
        
        # REQUÊTE SQL basée
        query = """
        SELECT h.nom, h.ville, s.nom_service, s.lits_disponibles 
        FROM service s
        JOIN hospital h ON s.hospital_id = h.id
        WHERE s.lits_disponibles > 0
        ORDER BY h.ville, h.nom
        """
        
        c.execute(query)
        results = c.fetchall()
        conn.close()

        if not results: 
            return "ALERTE: AUCUN LIT DISPONIBLE DANS LE RÉSEAU."
        
        txt = "DISPONIBILITÉS EN TEMPS RÉEL :\n"
        current_city = ""
        
        for nom_hopital, ville, service, lits in results:
            if ville != current_city:
                txt += f"\n SECTEUR {ville.upper()} :\n"
                current_city = ville
            txt += f"- {nom_hopital} : {service} ({lits} places)\n"
            
        return txt
        
    except Exception as e:
        return f"Erreur SQL : {e}"

In [4]:
api_key = os.environ.get('GROQ_API_KEY')
llm = ChatGroq(model="openai/gpt-oss-120b", api_key=api_key, temperature=0)

sql_agent = create_agent(
    model = llm,
    tools = [check_beds_availability],
    system_prompt = SYSTEME_PROMPTE_TEST,
    checkpointer = InMemorySaver(),
)
config = {"configurable": {"thread_id": "1"}}

In [5]:
q =HumanMessage(content = 'Moi c\'est Bilal')
response = sql_agent.invoke({'messages':[q]}, config)

In [6]:
print(response['messages'][-1].content)

{
  "message": "Bonjour Bilal, ravi de faire votre connaissance ! Je suis ici pour vous aider en cas d’urgence médicale ou de besoin de triage selon le protocole d’orientation des urgences du Togo. Si vous avez une question ou un problème de santé, n’hésitez pas à me le décrire afin que je puisse vous orienter vers la structure la plus adaptée. Sinon, je reste à votre disposition pour toute autre information non médicale."
}


In [7]:
q =HumanMessage(content = 'comment je m\'appele ?')
response = sql_agent.invoke({'messages':[q]}, config)
print(response['messages'][-1].content)

{
  "message": "Vous vous appelez Bilal."
}


In [12]:
# @tool
# def pharma_scrapping():


In [8]:
import requests
from bs4 import BeautifulSoup

URL = "https://sites.google.com/view/pharmaciedegarde-lome/tour-de-garde/"

html = requests.get(URL, timeout=15).text
soup = BeautifulSoup(html, "html.parser")

pharmacies_raw = []
infos = soup.get_text(separator="\n")

for a in soup.find_all("a"):
    href = a.get("href", "")
    text = a.get_text(strip=True).lower()
    

    if "maps" in href or "itin" in text:
        pharmacies_raw.append({
            "lien_itineraire": href,
            "context": a.find_parent().get_text(separator="\n")
        })

In [9]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["infos"],
    template =
            """
                Tu es un extracteur de données.

                À partir du texte ci-dessous (site officiel des pharmacies de garde à Lomé),
                extrais UNIQUEMENT les informations suivantes :

                Pour chaque pharmacie :
                - nom
                - quartier
                - emplacement
                - téléphone
                le lien des itineraires est deja fournie de l'invente pas

                Retourne STRICTEMENT un JSON valide
                Sans commentaire
                Sans texte supplémentaire

                Texte : {infos}
            """
)


In [10]:
from langchain_classic.chains import LLMChain
chain = LLMChain(llm = llm, prompt = prompt)

json_output = chain.run(infos)
print(json_output)

C:\Users\soulemane\AppData\Local\Temp\ipykernel_18932\2235372286.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm = llm, prompt = prompt)
C:\Users\soulemane\AppData\Local\Temp\ipykernel_18932\2235372286.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  json_output = chain.run(infos)


[
  {
    "nom": "PHARMACIE ADIDOGOME",
    "quartier": "Adidogomé",
    "emplacement": "Face au camp 2 ème RI d'adidogomé",
    "telephone": "91 05 78 21"
  },
  {
    "nom": "PHARMACIE LAUS DEO",
    "quartier": "Adidoadin",
    "emplacement": "Rte de Léo 2000, face Clinique Besthesda",
    "telephone": "93 00 65 75"
  },
  {
    "nom": "PHARMACIE OSSAN",
    "quartier": "Avédji",
    "emplacement": "Carrefour Avedji, face Ets LA LIMOUSINE",
    "telephone": "70 40 44 55"
  },
  {
    "nom": "PHARMACIE APOLLON",
    "quartier": "Avédji",
    "emplacement": "Face complexe scolaire Makafui - non du carrefour des hirondelles",
    "telephone": "93 50 42 55"
  },
  {
    "nom": "PHARMACIE DIEUDONNE",
    "quartier": "agoè Téléssou",
    "emplacement": "Route de léo 2000, non loin de FUCEC Agoè-Téléssou",
    "telephone": "70 44 84 59"
  },
  {
    "nom": "PHARMACIE VIGUEUR",
    "quartier": "Agbalpedogan",
    "emplacement": "Rue 267, AGBALPEDOGAN, Kilimandjaro",
    "telephone": "70 44 

In [11]:
prompt_2 = PromptTemplate(
    input_variables=["context", "url"],
    template="""
                Tu es un extracteur strict de données.

                À partir du texte ci-dessous, identifie UNE pharmacie et extrais :
                - nom
                - quartier
                - emplacement
                - telephone

                Le lien d'itinéraire est déjà fourni, ne l'invente pas.

                Retourne STRICTEMENT un JSON valide.

                Texte : {context}

                Lien itinéraire : {url}
            """
)

chain_2 = LLMChain(llm=llm, prompt=prompt_2)

results = []

for p in pharmacies_raw:
    out = chain_2.run(
        context=p["context"],
        url=p["lien_itineraire"]
    )
    results.append(out)


In [12]:
pharmacies_raw

[{'lien_itineraire': 'https://www.google.com/maps/dir//Pharmacie+d%27Adidogom%C3%A9,+Lom%C3%A9/@6.1850666,1.163665,17z/data=!4m8!4m7!1m0!1m5!1m1!1s0x102159051c625d73:0xc9b95e8e87d4b34d!2m2!1d1.1662806!2d6.1850232?entry=ttu',
  'context': 'Itinénaire'},
 {'lien_itineraire': 'https://www.google.com/maps/dir//654M%2BR6H+Pharmacie+Laus+D%C3%A9o,+Lom%C3%A9/@6.2070683,1.1805496,17z/data=!4m8!4m7!1m0!1m5!1m1!1s0x1021584f81d9b85d:0x242212c7d3c369e2!2m2!1d1.1831221!2d6.2070508?entry=ttu',
  'context': 'Itinénaire'},
 {'lien_itineraire': 'https://www.google.com/maps/dir//Pharmacie+Ossan,+Lom%C3%A9/@6.2011373,1.1781887,17z/data=!4m8!4m7!1m0!1m5!1m1!1s0x102158511d29ea4f:0x44058ce64a5a775f!2m2!1d1.1807433!2d6.2010802?entry=ttu',
  'context': 'Itinénaire'},
 {'lien_itineraire': 'https://www.google.com/maps/dir//Pharmacie+Apollon,+Unnamed+Road,+Lom%C3%A9/@6.2028583,1.1641086,17z/data=!4m8!4m7!1m0!1m5!1m1!1s0x102159ae4b9aac0f:0x5d7767cd4eb02bb8!2m2!1d1.1666778!2d6.2028463?entry=ttu',
  'context': 'Iti